# In this notebook, we're going to cover some of the most fundamental concepts of tensors using Tensorflow


More specfically, we're going to cover:
* Intro to tensors
* Getting info from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself


# Introduction to Tensors

In [2]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.8.2


In [3]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check the number of dimensions of a tensor
scalar.ndim

0

In [5]:
# Create a vector
vector = tf.constant([10, 10])
vector2 = tf.constant([[10, 10]])

vector, vector2, vector.ndim, vector2.ndim

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>,
 <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[10, 10]], dtype=int32)>,
 1,
 2)

In [6]:
# Create a Matrix
matrix = tf.constant([[10, 7], [7, 10]])
matrix, matrix.shape, matrix.ndim

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 7, 10]], dtype=int32)>, TensorShape([2, 2]), 2)

In [7]:
# Create another matrix (specify dtype)
matrix2 = tf.constant([[10., 7.], [3., 2.], [8., 9.]], dtype=tf.float16)

matrix2, matrix2.shape, matrix2.ndim

(<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
 array([[10.,  7.],
        [ 3.,  2.],
        [ 8.,  9.]], dtype=float16)>, TensorShape([3, 2]), 2)

In [8]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3], [4, 5, 6]],
                      [[7, 8, 9], [10, 11, 12]],
                      [[13, 14, 15], [16, 17, 18]]])
tensor, tensor.shape, tensor.ndim

(<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
 array([[[ 1,  2,  3],
         [ 4,  5,  6]],
 
        [[ 7,  8,  9],
         [10, 11, 12]],
 
        [[13, 14, 15],
         [16, 17, 18]]], dtype=int32)>, TensorShape([3, 2, 3]), 3)

What we've created so far:

* Scalar: a single number
* Vector: a number with direction
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (n can be any integer >= 0)

### Creating tensors with `tf.Variable`

In [9]:
# Create the same tensor with tf.Variable() as above (c for changeable, uc for unchangeable)
c_tensor = tf.Variable([10, 7])
uc_tensor = tf.constant([10, 7])

c_tensor, uc_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [10]:
# Let's try changing an element in c_tensor
# c_tensor[0] = 7
c_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>

In [11]:
# How about try .assign
c_tensor[0].assign(7)
c_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [12]:
# Let's try changing uc_tensor
# uc_tensor[0].assign(7)
uc_tensor

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>

### Creating Random Tensors

Random tensors are tensors of some arbitrary size of random numbers.

In [13]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1==random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.8757901 , -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.2878567 ],
        [-0.8757901 , -0.08857017],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [14]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]]
                           )
# Shuffling tensor across the first dimension
tf.random.set_seed(42)
not_shuffled = tf.random.shuffle(not_shuffled, seed=42) # as opposed to Dan's lecture, seems I have to re-assign for shuffle to work...

not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

** Figure out the difference between global-level and operation-level seeding... **

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

In [15]:
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [16]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [17]:
# Create a tensor of all zeroes
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing).

In [18]:
# Convert numpy array to tensor array
import numpy as np
numpy_A =  np.arange(1, 25, dtype=np.int32)
numpy_A
# capital for matrix or tensor
# non-capital for vector (scalar?)


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [19]:
A = tf.constant(numpy_A, shape=(3, 8))
B = tf.constant(numpy_A)
C = tf.constant(numpy_A, shape=(2, 3, 4))
# D = tf.constant(numpy_A, shape=(2, 3, 5)) # will throw error (different number of elements..)

A, B, C

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>)

### Getting Info From Tensors

* Shape
* Rank
* Axis or dimension
* Size

In [20]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape = [2, 3, 4, 5])

rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [21]:
# get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along 0 axis:", rank_4_tensor.shape[0])
print("Elements along last axis:", rank_4_tensor.shape[-1]) # -1 to get last index
print("Size of tensor:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along 0 axis: 2
Elements along last axis: 5
Size of tensor: 120


### Indexing Tensors
Tensors can be index just like Python lists.

In [22]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [23]:
# Get first element from each dimension from each index except the final one
rank_4_tensor.shape,  rank_4_tensor[:1, :1, :, :1]

(TensorShape([2, 3, 4, 5]),
 <tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
 array([[[[0.],
          [0.],
          [0.],
          [0.]]]], dtype=float32)>)

In [24]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7], [3, 4]])

rank_2_tensor, rank_2_tensor.shape, rank_2_tensor.ndim

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4]], dtype=int32)>, TensorShape([2, 2]), 2)

In [25]:
rank_4_tensor[-1, :, :, -1]

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [26]:
rank_2_tensor[-1, -1]

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [27]:
# Add an extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]# " ... " means every axis prior (same as ":, :,")
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [28]:
# Alternative to tf.newaxis
expand1 = tf.expand_dims(rank_2_tensor, axis = -1)
expand1, expand1.shape

(<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
 array([[[10],
         [ 7]],
 
        [[ 3],
         [ 4]]], dtype=int32)>, TensorShape([2, 2, 1]))

In [29]:
# Expanding the 0-axis
expand2 = tf.expand_dims(rank_2_tensor, axis = 0)
expand2, expand2.shape

(<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
 array([[[10,  7],
         [ 3,  4]]], dtype=int32)>, TensorShape([1, 2, 2]))

### Manipulating tensors (tensor operations)
**Basic Operations**
`+`, `-`, `*`, `/`

In [30]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])


In [31]:
tensor + 10, tensor *10, tf.multiply(tensor, 10)

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[20, 17],
        [13, 14]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  70],
        [ 30,  40]], dtype=int32)>, <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  70],
        [ 30,  40]], dtype=int32)>)

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are two rules our tensors need to fulfill if we're going to matrix multiply them:

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [32]:
# Matrix multiplication in tensorflow
tf.matmul(tensor, tensor), tensor * tensor 

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[121,  98],
        [ 42,  37]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  49],
        [  9,  16]], dtype=int32)>)

In [33]:
# Matrix multiplication with Python operator @
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [34]:
tensor.shape

TensorShape([2, 2])

In [35]:
# create a (3, 2) tensor  
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
# create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [36]:
# Try to matrix multiply tensors of same shape 
# will throw errors
# X @ Y, tf.matmul(X, Y), 



In [37]:
# Let's change the shape of Y
X.shape, tf.reshape(Y, shape=(2, 3)).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [38]:
# try to multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [39]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [40]:
tf.matmul(tf.reshape(X, shape=(2, 3)), Y), tf.matmul(Y, tf.reshape(X, shape=(2, 3)))

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[ 58,  64],
        [139, 154]], dtype=int32)>,
 <tf.Tensor: shape=(3, 3), dtype=int32, numpy=
 array([[ 39,  54,  69],
        [ 49,  68,  87],
        [ 59,  82, 105]], dtype=int32)>)

In [41]:
# Can do the same with transpose (transpose flips across diagonal unlike reshape which suffles elements around into the desired shape)
tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [42]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The Dot Product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:

*  `tf.matmul()`

* `tf.tensordot()`

In [43]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
''' NOT understanding this, come back... '''
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [44]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [45]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

### Changing the datatype of a Tensor

In [46]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [47]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [48]:
# 32-bit precision is default in TensorFlow
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [49]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

In [50]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

### Aggregating Tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [51]:
# Get the absolute values
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [52]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:

* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [53]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([ 5,  1, 66, 43, 29, 50, 22, 37, 93, 94, 56, 85, 53, 87, 67, 30, 75,
       38, 74, 32, 45, 55, 15, 91, 37,  2, 67, 61, 39, 35,  1,  9, 14, 43,
       42, 84,  7, 71, 24, 18,  7, 92, 82, 99, 41, 55,  4, 82, 70, 98])>

In [54]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [55]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [56]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [57]:
# find the mean 
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=48>

In [58]:
# find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2427>

In [59]:
#  Find the variance of our tensor

import tensorflow_probability as tfp # need access to this library to get variance
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=892>

In [60]:
# Find the standard deviation of a tensor
tf.math.reduce_std(tf.cast(E, dtype=tf.float32)) # doesn't seem to take integers... why??

<tf.Tensor: shape=(), dtype=float32, numpy=29.869858>

In [61]:
# Find the variance of E tensor
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=892.20844>

### Find the Positional Max and Min

In [62]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [63]:
# Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [64]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [65]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [66]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [67]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [68]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions)

In [69]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [70]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [71]:
# "tf.squeeze" will remove all dimensions of 1 from tensor, "squeezing" it.
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>, TensorShape([50]))

### One-Hot Encoding

In [72]:
# Create a list of indices
some_list = [0, 1, 1, 10] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=20)

<tf.Tensor: shape=(4, 20), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)>

In [73]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="yo I love deep learning", off_value="I also like to dance")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'yo I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'yo I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'I also like to dance']], dtype=object)>

### Squaring, log, square root

In [74]:
# Create a new tensor
H = tf.range(1, 10)
tf

<module 'tensorflow' from '/usr/local/lib/python3.7/dist-packages/tensorflow/__init__.py'>

In [75]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [76]:
# Square-root it (method requires a non-int type)
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [77]:
# Find the log (method requires a non-int type)
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy

TensorFlow interacts beautifully with NumpPy arrays.

In [78]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [79]:
# Covnert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [80]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [81]:
J = tf.constant([3.])
J

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.], dtype=float32)>

In [82]:
J.numpy()

array([3.], dtype=float32)

In [83]:
J.numpy()[0]

3.0

In [84]:
# The default types of each are slightly different 
numpy_J = tf.constant(np.array([3., 7., 10.])) # numpy are float64...
tensor_J = tf.constant([3., 7., 10.]) # tensors are float32
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype


(tf.float64, tf.float32)

### Finding access to GPUs

In [85]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [86]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [87]:
!nvidia-smi

Thu Jul 14 21:58:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    29W /  70W |    286MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible.